In [ ]:
import pip

!pip install Pillow

In [2]:
"""
pip install opencv-contrib-python==3.4.2.16
"""
import base64
import json
import pprint
import cv2
import numpy as np
from finalProject.classes.yolo import Yolo
from finalProject.utils.drawing.draw import draw_final_results
from finalProject.utils.images.imagesUtils import Image
from finalProject.utils.keyPoints.AlgoritamKeyPoints import create_keypoints_descriptors, sift_keypoints_detection
from finalProject.utils.matchers.Matchers import compare_between_descriptors, flann_matcher
from finalProject.utils.preprocessing.preprocess import read_frames_from_video, is_frames_exists, reduce_noise, removeRemovalColor
from finalProject.utils.tracking.TrackingByYolo import source_detection_by_yolo, tracking_by_yolo
import matplotlib.pyplot as plt
import random
import pandas as pd
import io
from PIL import Image
import os


def sobel(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    sobel_intensity = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx, 2.0),
                                               1.0, cv2.pow(sobely, 2.0), 1.0, 0.0))
    return sobel_intensity


def sobel_keypoints(image):
    sobelImage = sobel(image)
    # norm
    image8bit = cv2.normalize(sobelImage, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    k, d = sift_keypoints_detection(image8bit)
    return k, d, image8bit


def forward(frames):
    outputs = []
    for frame in frames:
        frame.gray_image = cv2.cvtColor(frame.frame_image, cv2.COLOR_BGR2GRAY)
        frame.gray_image = sobel(frame.gray_image)
        key_des_image = sobel_keypoints(frame.gray_image)
        outputs.append(key_des_image)
    return outputs


def saveImage(image, folderName):
    rand = random.uniform(0, 1)
    imageName = "image_"+str(rand)
    path = "./reports/"+str(folderName)+"/"+imageName+".png"
    cv2.imwrite(path, image)
    return imageName


def convert(folderName):
    def convertImagetoIds(outputs):
        outputs[0] = saveImage(outputs[0],folderName)
        outputs[1] = saveImage(outputs[1],folderName)
        outputs[2] = saveImage(outputs[2],folderName)
        return outputs
    return convertImagetoIds


def exportToExcel(outputs):
    rand = random.uniform(0, 1)  # random folder number
    folderName = "folderReport_"+str(rand)

    path = "./reports/"+folderName
    os.makedirs(path, mode=0o777, exist_ok=False)

    outputs = list(map(convert(folderName),outputs))
    df2 = pd.DataFrame(np.array(outputs), 
                       columns=['outputImage', 'img1', 'img2', 'len(k1)', 'len(k2)', 'len(match)', 'accuracy'])
    df2.to_excel("./reports/"+folderName+"/report_"+folderName+".xls")


def cross_correction(key_des_image_source, key_des_image_target, threshold=0.8):
    (k1, d1, img1) = key_des_image_source
    (k2, d2, img2) = key_des_image_target
    match = flann_matcher(d1, d2, threshold=threshold)
    if len(match) == 0:
        return None
     
    output = cv2.drawMatchesKnn(img1, k1, img2, k2, match, outImg=None)
    acc = len(match)/len(k1)
    return [output, img1, img2, len(k1), len(k2), len(match), min(acc, 1)]  # item = (outputs,img1,len(k1),img2,len(k2),acc..)


def plotSquare(images, _titles, threshold, each_column=10):
    number_images = len(images)
    rows = (number_images // each_column)
    index = 1

    print(rows)
    fig, axes = plt.subplots(rows, each_column, figsize=(48, 24),dpi=300)
    fig.subplots_adjust(hspace=0.3, wspace=0.05)

    for ax,image,title in zip(axes.flat, images, _titles):
        ax.imshow(image)
        ax.set_title(title)
        index += 1

    rand = random.uniform(0, 1)
    pathX = "./figures/fig_test"+str(rand)+"_thresh_"+str(threshold)+".svg"
    print("saving on", pathX)
    plt.savefig(pathX, dpi=fig.dpi)
    plt.clf()
    plt.close()

In [3]:
# init yolo
yolo = Yolo()
yolo.initYolo()
pp = pprint.PrettyPrinter(indent=4)


[INFO] loading YOLO from disk...


In [4]:
def run(threshold=0.8):
  
    with open('./config.txt') as file_json:
        config = json.load(file_json)

        # source
        source_frames = read_frames_from_video(config["source"])
        if not is_frames_exists(source_frames):
            print("problem with source video input")
            exit(0)

        source_person = source_detection_by_yolo(source_frames, yolo,
                                                 is_video=config["source"]["isVideo"],
                                            config=config["source"])
        if source_person is None:
            print("fail to detect human on source video")
            exit(0)

        # target
        target_frames = read_frames_from_video(config["target"])
        if not is_frames_exists(target_frames):
            print("problem with target video input")
            exit(0)

        if not is_frames_exists(target_frames):
            print("problem with target video input - reduce noise")
            exit(0)

        target_people = tracking_by_yolo(target_frames, yolo, is_video=config["target"]["isVideo"], config=config["target"])

        cv2.destroyAllWindows()
        
        # source
        sources_output = forward(source_person.frames)  # array of outputs (key,des,image)
        targets_outputs = []  # array of targets , each target have array of frames
        for target_person in target_people:
            target_output = forward(target_person.frames)  # array of frames
            targets_outputs.append(target_output)

        outputs = []  # each frame

        for source_frame in sources_output:
            for target in targets_outputs:
                for target_frame in target:  # threshold for match ! bigger - more lines
                    cross = cross_correction(source_frame, target_frame, threshold)
                    if cross is not None:
                        outputs.append(cross)

        exportToExcel(outputs)
                        
#         titles = ["test"] * (len(plots) - 1)
#         plotSquare(plots, titles,threshold)
#         print("finish")

In [5]:
for thresh in [0.8]:
    run(threshold=thresh)
